# Week 4 -- Data Cleaning

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re
import warnings
warnings.simplefilter('ignore')

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.1f' % x)

## Functions

In [3]:
def check_names(df1, df2):
    """
    This funciton compares the list of names from one dataframe to another and updates where the names do not match 
    so they will merge correctly.
    
    Parameters:
        df1: first dataframe to compare to; this should be the dataframe on the left that you will join with when 
        they merge.
        df2: second dataframe, should be the dataframe on the right for the later merge.
        
    Returns:
        Updated name list for df2 that should match df1.
    """
    
    df_players = list(df1['Player'])    
    for i in range(len(df2['Player'])):
        name = df2['Player'][i]
        split_name = name.split(' ')
        if (split_name[0] + ' ' + split_name[1]) in df_players:
            df2['Player'][i] = split_name[0] + ' ' + split_name[1]
        else:
            df2['Player'][i] = name

In [4]:
def name_update(dataframe, column):
    """
    This function updates a column in a dataframe by stripping any excess spaces surrounding the observation.
    
    Parameters:
        dataframe: the dataframe you want to update.
        column: the column you want to update.
    
    Returns:
        Updated dataframe column.    
    """
    dataframe[column] = dataframe[column].map(lambda x: x.strip())

In [5]:
def update_int_dtype(dataframe, column_list):
    """
    This function updates the data type for a list of columns to integer.
    
    Parameters:
        dataframe: the dataframe you want to update.
        column_list: list of columns to iterate through.
        
    Returns:
        Updated dataframe column data types to integer.   
    """
    for column in column_list:
        dataframe[column] = dataframe[column].map(lambda x: int(x))

In [6]:
def update_float_dtype(dataframe, column_list):
    """
    This function takes a dataframe object and converts it to a float.
    
    Parameters:
        dataframe: the dataframe you want to update.
        column_list: list of columns to iterate through.
    
    Returns:
        Updated dataframe column data types to a float.  
    """
    for column in column_list:
        for i in range(len(dataframe[column])):
            item = dataframe[column][i]
            if item == '':
                dataframe[column][i] = '0.0'
            else:
                dataframe[column][i] = item 
            updated_item = str(dataframe[column][i])
            ones, tenths = updated_item.split('.')
            ones = int(ones)
            tenths = int(tenths) * .1
            dataframe[column][i] = ones + tenths
        dataframe[column] = dataframe[column].map(lambda x: float(x))

In [7]:
def fill_blanks(dataframe, column_list):
    """
    This function fills blank values with 0. This should be used for a column that will be an integer.
    
    Parameters:
        dataframe: the dataframe you want to update.
        column_list: list of columns to iterate through.    
    
    Returns:
        Updated dataframe columns with filled values.      
    """
    for column in column_list:
        for i in range(len(dataframe[column])):
            item = dataframe[column][i]
            if item == '':
                dataframe[column][i] = 0
            else:
                dataframe[column][i] = item

In [8]:
def remove_comma(dataframe, column_list):
    """
    This function removes the comma from a value in a column.
    
    Parameters:
        dataframe: the dataframe you want to update.
        column_list: list of columns to iterate through.
        
    Returns:
        Updated dataframe columns less any commas that might appear.    
    """  
    for column in column_list:
        for i in range(len(dataframe[column])):
            item = dataframe[column][i]
            if len(item.split(',')) > 1:
                one, two = item.split(',')
                dataframe[column][i] = one + two

In [9]:
def adjust_float(dataframe, column_list):
    """
    This function adjusts values in a column to be workable with the 'update_float_dtype' function.
    
    Parameters:
        dataframe: the dataframe you want to update.
        column_list: list of columns to iterate through.
        
    Returns:
        Updated dataframe columns with objects with a decimal point value behind it.    
    """
    for column in column_list:
        for i in range(len(dataframe[column])):
            item = dataframe[column][i]
            if len(item) < 3:
                dataframe[column][i] = item + '.0'
            else:
                dataframe[column][i] = item

In [10]:
def update_percent(dataframe, column):
    """
    This function updates a column with a percent sign so it can be manipulated into a float with the 
    'update_float_dtype' function.
    
    Parameters:
       dataframe: the dataframe you want to update.
        column_list: list of columns to iterate through.
        
    Returns:
        Updated dataframe column with no percent sign and added decimal point where needed.     
    """
    dataframe[column] = dataframe[column].map(lambda x: x.strip('%'))
    for i in range(len(dataframe[column])):
        item = dataframe[column][i]
        if len(item) < 3:
            dataframe[column][i] = item + '.0'
        else:
            dataframe[column][i] = item

## Read in Collected Data

In [11]:
df = pd.read_pickle('player_stats')
fantasy_pts = pd.read_pickle('fantasy_weeks')
defense = pd.read_pickle('defense_data')

## Merge Last Week Fantasy Pts with this Week

In [12]:
no_games = 4

In [13]:
lastwk = pd.read_pickle('fantweeks_1_' + str(no_games - 1))
lastwk.head()

,Player,Team,Position,TTL,Week_1,Week_2,Week_3
0,Russell Wilson,SEA,QB,103.0,31.8,34.4,36.8
1,Josh Allen,BUF,QB,94.9,28.2,34.5,32.2
2,Patrick Mahomes II,KC,QB,87.9,20.4,27.5,40.0
3,Dak Prescott,DAL,QB,86.9,17.6,39.8,29.5
4,Kyler Murray,ARI,QB,85.1,27.3,33.1,24.7


In [14]:
fantasy_pts = pd.merge(fantasy_pts, lastwk, 'left', on='Player')
fantasy_pts.head()

,Player,Team_x,Position_x,TTL_x,Week_4,Team_y,Position_y,TTL_y,Week_1,Week_2,Week_3
0,Dak Prescott,DAL,QB,125.2,38.3,DAL,QB,86.9,17.6,39.8,29.5
1,Russell Wilson,SEA,QB,124.9,21.9,SEA,QB,103.0,31.8,34.4,36.8
2,Josh Allen,BUF,QB,120.3,25.4,BUF,QB,94.9,28.2,34.5,32.2
3,Kyler Murray,ARI,QB,108.2,23.1,ARI,QB,85.1,27.3,33.1,24.7
4,Patrick Mahomes II,KC,QB,108.1,20.2,KC,QB,87.9,20.4,27.5,40.0


In [16]:
fantasy_pts.drop(columns=['Team_y', 'Position_y', 'TTL_y'], inplace=True)
fantasy_pts.rename(columns = {'Team_x': 'Team', 'Position_x': 'Position', 'TTL_x': 'TTL'}, inplace = True)
fantasy_pts.columns

Index(['Player', 'Team', 'Position', 'TTL', 'Week_4', 'Week_1', 'Week_2',
       'Week_3'],
      dtype='object')

In [17]:
fantasy_pts = fantasy_pts[['Player', 'Team', 'Position', 'TTL', 'Week_1', 'Week_2', 'Week_3', 'Week_4']]
fantasy_pts.head()

,Player,Team,Position,TTL,Week_1,Week_2,Week_3,Week_4
0,Dak Prescott,DAL,QB,125.2,17.6,39.8,29.5,38.3
1,Russell Wilson,SEA,QB,124.9,31.8,34.4,36.8,21.9
2,Josh Allen,BUF,QB,120.3,28.2,34.5,32.2,25.4
3,Kyler Murray,ARI,QB,108.2,27.3,33.1,24.7,23.1
4,Patrick Mahomes II,KC,QB,108.1,20.4,27.5,40.0,20.2


In [18]:
print(fantasy_pts.shape)

(582, 8)


### Pickle Combined DataFrame for Next Week

In [19]:
fantasy_pts.to_pickle('fantweeks_1_' + str(no_games))

## Merge Offensive Players & Weekly Fantasy Pts

In [20]:
print(df.shape)
df.head()

(488, 28)


,Player,Team,Position,Age,Games,GamesStarted,CompletedPasses,PassesAttempted,PassingYds,PassingTDs,Interceptions,RushingAttempts,RushingYds,RushingYdspAtt,RushingTDs,Targeted,Receptions,ReceivingYds,YdspReception,ReceivingTDs,Fumbles,LostFumbles,TtlTDs,TwoPTConversions,TwoPTConversionPasses,FDFantasyPts,PositionRank,OverallRank
0,Alvin Kamara,NOR,RB,25,4,3,0,0,0,0,0,50,236,4.72,4,35,30,321,10.70,3,0,0,7,,,112.7,1,1
1,Russell Wilson,SEA,QB,32,4,4,103,137,1285,16,2,18,95,5.28,0,0,0,0,,0,1,0,0,,1,124.9,1,2
2,Dak Prescott,DAL,QB,27,4,4,137,201,1690,9,3,16,86,5.38,3,0,0,0,,0,3,3,3,1,1,125.2,2,3
3,Aaron Jones,GNB,RB,26,4,4,0,0,0,0,0,65,374,5.75,4,23,15,135,9.00,2,1,0,6,,,94.4,2,4
4,Dalvin Cook,MIN,RB,25,4,4,0,0,0,0,0,75,424,5.65,6,11,7,40,5.71,0,1,1,6,3,,89.9,3,5


In [21]:
print(fantasy_pts.shape)
fantasy_pts.head()

(582, 8)


,Player,Team,Position,TTL,Week_1,Week_2,Week_3,Week_4
0,Dak Prescott,DAL,QB,125.2,17.6,39.8,29.5,38.3
1,Russell Wilson,SEA,QB,124.9,31.8,34.4,36.8,21.9
2,Josh Allen,BUF,QB,120.3,28.2,34.5,32.2,25.4
3,Kyler Murray,ARI,QB,108.2,27.3,33.1,24.7,23.1
4,Patrick Mahomes II,KC,QB,108.1,20.4,27.5,40.0,20.2


In [22]:
check_names(df, fantasy_pts)

In [23]:
name_update(df, 'Player')
name_update(fantasy_pts, 'Player')

In [24]:
offense = pd.merge(df, fantasy_pts, 'left', on='Player')

In [25]:
offense.drop(columns = ['Team_y', 'Position_y'], inplace = True)
offense.rename(columns = {'Team_x': 'Team', 'Position_x': 'Position'}, inplace = True)

In [26]:
print(offense.shape)
offense.head(10)

(488, 33)


,Player,Team,Position,Age,Games,GamesStarted,CompletedPasses,PassesAttempted,PassingYds,PassingTDs,Interceptions,RushingAttempts,RushingYds,RushingYdspAtt,RushingTDs,Targeted,Receptions,ReceivingYds,YdspReception,ReceivingTDs,Fumbles,LostFumbles,TtlTDs,TwoPTConversions,TwoPTConversionPasses,FDFantasyPts,PositionRank,OverallRank,TTL,Week_1,Week_2,Week_3,Week_4
0,Alvin Kamara,NOR,RB,25,4,3,0,0,0,0,0,50,236,4.72,4,35,30,321,10.70,3,0,0,7,,,112.7,1,1,97.7,18.7,29.4,31.7,17.9
1,Russell Wilson,SEA,QB,32,4,4,103,137,1285,16,2,18,95,5.28,0,0,0,0,,0,1,0,0,,1,124.9,1,2,124.9,31.8,34.4,36.8,21.9
2,Dak Prescott,DAL,QB,27,4,4,137,201,1690,9,3,16,86,5.38,3,0,0,0,,0,3,3,3,1,1,125.2,2,3,125.2,17.6,39.8,29.5,38.3
3,Aaron Jones,GNB,RB,26,4,4,0,0,0,0,0,65,374,5.75,4,23,15,135,9.00,2,1,0,6,,,94.4,2,4,86.9,13.6,41.6,14.6,17.1
4,Dalvin Cook,MIN,RB,25,4,4,0,0,0,0,0,75,424,5.65,6,11,7,40,5.71,0,1,1,6,3,,89.9,3,5,86.4,20.8,15.1,23.9,26.6
5,Josh Allen,BUF,QB,24,4,4,105,148,1326,12,1,25,83,3.32,3,0,0,0,,0,3,3,3,,,120.3,3,6,120.3,28.2,34.5,32.2,25.4
6,Patrick Mahomes,KAN,QB,25,4,4,101,150,1134,11,0,18,108,6.00,1,0,0,0,,0,0,0,1,,1,108.2,4,7,108.1,20.4,27.5,40.0,20.2
7,Kyler Murray,ARI,QB,23,4,4,99,144,919,7,5,32,265,8.28,4,0,0,0,,0,1,1,4,,,108.3,5,8,108.2,27.3,33.1,24.7,23.1
8,Aaron Rodgers,GNB,QB,37,4,4,98,139,1214,13,0,6,31,5.17,0,1,1,-6,-6.00,0,0,0,0,,,103.6,6,9,103.1,30.8,18.2,24.5,29.6
9,Calvin Ridley,ATL,WR,26,4,4,0,0,0,0,0,2,6,3.00,0,40,21,349,16.62,4,0,0,4,,,70.0,1,10,59.5,24.9,22.9,11.7,0.0


In [27]:
offense.isnull().sum()

Player                     0
Team                       0
Position                   0
Age                        0
Games                      0
GamesStarted               0
CompletedPasses            0
PassesAttempted            0
PassingYds                 0
PassingTDs                 0
Interceptions              0
RushingAttempts            0
RushingYds                 0
RushingYdspAtt             0
RushingTDs                 0
Targeted                   0
Receptions                 0
ReceivingYds               0
YdspReception              0
ReceivingTDs               0
Fumbles                    0
LostFumbles                0
TtlTDs                     0
TwoPTConversions           0
TwoPTConversionPasses      0
FDFantasyPts               0
PositionRank               0
OverallRank                0
TTL                       10
Week_1                    83
Week_2                    85
Week_3                    88
Week_4                   114
dtype: int64

In [28]:
offense.fillna('0.0', inplace = True)

In [29]:
off_integers = ['Age', 'Games', 'GamesStarted', 'CompletedPasses', 'PassesAttempted', 'PassingYds', 'PassingTDs', 
            'Interceptions', 'RushingAttempts', 'RushingYds', 'RushingTDs', 'Targeted', 'Receptions', 
            'ReceivingYds', 'ReceivingTDs', 'Fumbles', 'LostFumbles', 'TtlTDs']

off_floats = ['RushingYdspAtt', 'YdspReception', 'FDFantasyPts', 'TTL','Week_1', 'Week_2', 'Week_3']

In [30]:
update_int_dtype(offense, off_integers)

In [31]:
two_pts = ['TwoPTConversions', 'TwoPTConversionPasses']
fill_blanks(offense, two_pts)
update_int_dtype(offense, two_pts)

In [32]:
update_float_dtype(offense, off_floats)

In [33]:
offense.head()

,Player,Team,Position,Age,Games,GamesStarted,CompletedPasses,PassesAttempted,PassingYds,PassingTDs,Interceptions,RushingAttempts,RushingYds,RushingYdspAtt,RushingTDs,Targeted,Receptions,ReceivingYds,YdspReception,ReceivingTDs,Fumbles,LostFumbles,TtlTDs,TwoPTConversions,TwoPTConversionPasses,FDFantasyPts,PositionRank,OverallRank,TTL,Week_1,Week_2,Week_3,Week_4
0,Alvin Kamara,NOR,RB,25,4,3,0,0,0,0,0,50,236,11.2,4,35,30,321,17.0,3,0,0,7,0,0,112.7,1,1,97.7,18.7,29.4,31.7,17.9
1,Russell Wilson,SEA,QB,32,4,4,103,137,1285,16,2,18,95,7.8,0,0,0,0,0.0,0,1,0,0,0,1,124.9,1,2,124.9,31.8,34.4,36.8,21.9
2,Dak Prescott,DAL,QB,27,4,4,137,201,1690,9,3,16,86,8.8,3,0,0,0,0.0,0,3,3,3,1,1,125.2,2,3,125.2,17.6,39.8,29.5,38.3
3,Aaron Jones,GNB,RB,26,4,4,0,0,0,0,0,65,374,12.5,4,23,15,135,9.0,2,1,0,6,0,0,94.4,2,4,86.9,13.6,41.6,14.6,17.1
4,Dalvin Cook,MIN,RB,25,4,4,0,0,0,0,0,75,424,11.5,6,11,7,40,12.1,0,1,1,6,3,0,89.9,3,5,86.4,20.8,15.1,23.9,26.6


In [34]:
offense.dtypes

Player                    object
Team                      object
Position                  object
Age                        int64
Games                      int64
GamesStarted               int64
CompletedPasses            int64
PassesAttempted            int64
PassingYds                 int64
PassingTDs                 int64
Interceptions              int64
RushingAttempts            int64
RushingYds                 int64
RushingYdspAtt           float64
RushingTDs                 int64
Targeted                   int64
Receptions                 int64
ReceivingYds               int64
YdspReception            float64
ReceivingTDs               int64
Fumbles                    int64
LostFumbles                int64
TtlTDs                     int64
TwoPTConversions           int64
TwoPTConversionPasses      int64
FDFantasyPts             float64
PositionRank              object
OverallRank               object
TTL                      float64
Week_1                   float64
Week_2    

## Defense Stats

In [35]:
defense.head()

,Team,Ttl_Pts_Allowed,Ttl_Offense_Plays_Allowed,Yds_p_Play,Ttl_Yds,Rushing_Att,Rushing_Yds,Rushing_Yds_p_Att,Rushing_TDs,Passing_Att,Passing_Yds_p_Att,Completions,Yds_p_Completion,Passing_Yds,Passing_TDs,RZ_Att,RZ_TD,RZ_Percent,Ttl_Turnovers,Interceptions,Fumbles,Sacks
0,Indianapolis Colts,56,213,4.4,945,86,308,3.6,1,117,5,73,9.6,637,5,8,5,62.5%,7,7,0,10
1,Pittsburgh Steelers,58,183,4.8,870,61,162,2.7,1,107,5.8,64,12.8,708,6,9,4,44.4%,5,4,1,15
2,Kansas City Chiefs,70,264,5.4,"1,424",122,644,5.3,3,130,5.5,77,11.1,780,4,13,7,53.8%,7,5,2,12
3,San Francisco 49ers,71,248,4.8,"1,179",108,443,4.1,3,132,5.3,82,9.5,736,3,6,3,50%,5,3,2,8
4,Baltimore Ravens,73,268,5.5,"1,470",97,390,4,3,162,6.3,109,10.7,"1,080",6,9,7,77.8%,7,2,5,9


In [136]:
def_fantasy = fantasy_pts[fantasy_pts['Position'] == 'DST']

In [137]:
defense_df = pd.merge(defense, def_fantasy, 'left', left_on = 'Team', right_on = 'Player')

In [138]:
defense_df.head()

,Team_x,Ttl_Pts_Allowed,Ttl_Offense_Plays_Allowed,Yds_p_Play,Ttl_Yds,Rushing_Att,Rushing_Yds,Rushing_Yds_p_Att,Rushing_TDs,Passing_Att,Passing_Yds_p_Att,Completions,Yds_p_Completion,Passing_Yds,Passing_TDs,RZ_Att,RZ_TD,RZ_Percent,Ttl_Turnovers,Interceptions,Fumbles,Sacks,Player,Team_y,Position,TTL,Week_1,Week_2,Week_3
0,Indianapolis Colts,46,154,4.4,676,70,280,4,1,75,6.1,47,9.7,454,4,6,4,66.7%,6,6,0,9,Indianapolis Colts,IND,DST,45.0,4.0,15.0,26.0
1,San Francisco 49ers,48,189,4.8,912,80,350,4.4,2,104,5.7,64,9.2,588,2,4,2,50%,4,2,2,5,San Francisco 49ers,SF,DST,21.0,4.0,5.0,12.0
2,Baltimore Ravens,59,125,4.9,610,44,189,4.3,0,75,6.2,46,10.1,464,2,3,2,66.7%,5,2,3,6,Baltimore Ravens,BAL,DST,31.0,15.0,15.0,1.0
3,Los Angeles Chargers,59,188,5.4,"1,011",71,328,4.6,1,111,6.6,72,10.1,730,3,9,2,22.2%,2,1,1,6,Los Angeles Chargers,LAC,DST,14.0,11.0,1.0,2.0
4,Pittsburgh Steelers,60,183,4.8,870,61,162,2.7,1,107,7.7,64,12.8,819,6,9,4,44.4%,5,4,1,15,Pittsburgh Steelers,PIT,DST,28.0,8.0,13.0,7.0


In [139]:
defense_df.drop(columns=['Player', 'Team_y', 'Position'], inplace=True)
defense_df.rename(columns={'Team_x': 'Team'}, inplace=True)

In [140]:
defense_df.head()

,Team,Ttl_Pts_Allowed,Ttl_Offense_Plays_Allowed,Yds_p_Play,Ttl_Yds,Rushing_Att,Rushing_Yds,Rushing_Yds_p_Att,Rushing_TDs,Passing_Att,Passing_Yds_p_Att,Completions,Yds_p_Completion,Passing_Yds,Passing_TDs,RZ_Att,RZ_TD,RZ_Percent,Ttl_Turnovers,Interceptions,Fumbles,Sacks,TTL,Week_1,Week_2,Week_3
0,Indianapolis Colts,46,154,4.4,676,70,280,4,1,75,6.1,47,9.7,454,4,6,4,66.7%,6,6,0,9,45.0,4.0,15.0,26.0
1,San Francisco 49ers,48,189,4.8,912,80,350,4.4,2,104,5.7,64,9.2,588,2,4,2,50%,4,2,2,5,21.0,4.0,5.0,12.0
2,Baltimore Ravens,59,125,4.9,610,44,189,4.3,0,75,6.2,46,10.1,464,2,3,2,66.7%,5,2,3,6,31.0,15.0,15.0,1.0
3,Los Angeles Chargers,59,188,5.4,"1,011",71,328,4.6,1,111,6.6,72,10.1,730,3,9,2,22.2%,2,1,1,6,14.0,11.0,1.0,2.0
4,Pittsburgh Steelers,60,183,4.8,870,61,162,2.7,1,107,7.7,64,12.8,819,6,9,4,44.4%,5,4,1,15,28.0,8.0,13.0,7.0


In [141]:
def_int = ['Ttl_Pts_Allowed', 'Ttl_Offense_Plays_Allowed', 'Ttl_Yds', 'Rushing_Att', 'Rushing_Yds', 'Rushing_TDs', 
           'Passing_Att', 'Completions', 'Passing_Yds', 'Passing_TDs', 'RZ_Att', 'RZ_TD', 'Ttl_Turnovers', 
           'Interceptions', 'Fumbles', 'Sacks']

def_floats = ['Yds_p_Play', 'Rushing_Yds_p_Att', 'Passing_Yds_p_Att', 'Yds_p_Completion', 'TTL', 'Week_1', 
              'Week_2', 'Week_3']

In [142]:
remove_comma(defense_df, def_int)

In [143]:
update_int_dtype(defense_df, def_int)

In [144]:
adjust_float(defense_df, def_floats)

In [145]:
update_float_dtype(defense_df, def_floats)

In [146]:
update_percent(defense_df, 'RZ_Percent')

In [147]:
rz = ['RZ_Percent']
update_float_dtype(defense_df, rz)

In [148]:
defense_df.head()

,Team,Ttl_Pts_Allowed,Ttl_Offense_Plays_Allowed,Yds_p_Play,Ttl_Yds,Rushing_Att,Rushing_Yds,Rushing_Yds_p_Att,Rushing_TDs,Passing_Att,Passing_Yds_p_Att,Completions,Yds_p_Completion,Passing_Yds,Passing_TDs,RZ_Att,RZ_TD,RZ_Percent,Ttl_Turnovers,Interceptions,Fumbles,Sacks,TTL,Week_1,Week_2,Week_3
0,Indianapolis Colts,46,154,4.4,676,70,280,4.0,1,75,6.1,47,9.7,454,4,6,4,66.7,6,6,0,9,45.0,4.0,15.0,26.0
1,San Francisco 49ers,48,189,4.8,912,80,350,4.4,2,104,5.7,64,9.2,588,2,4,2,50.0,4,2,2,5,21.0,4.0,5.0,12.0
2,Baltimore Ravens,59,125,4.9,610,44,189,4.3,0,75,6.2,46,10.1,464,2,3,2,66.7,5,2,3,6,31.0,15.0,15.0,1.0
3,Los Angeles Chargers,59,188,5.4,1011,71,328,4.6,1,111,6.6,72,10.1,730,3,9,2,22.2,2,1,1,6,14.0,11.0,1.0,2.0
4,Pittsburgh Steelers,60,183,4.8,870,61,162,2.7,1,107,7.7,64,12.8,819,6,9,4,44.4,5,4,1,15,28.0,8.0,13.0,7.0


## Pickle Cleaned DataFrames

In [150]:
offense.to_pickle('players')
defense_df.to_pickle('defense')